In [1]:
import streamlit as st
import datetime
from glob import glob
from PIL import Image
import numpy as np
import base64
import io
import sys

if '../app' not in sys.path:
    sys.path.append('../app/')

from utils.locate_timings import bin_confusion, confusion2transcript
from utils.identify_theme import send2chatgpt, split_confused_transcripts_by_line

In [2]:
d = input('Select a session date:')
d = datetime.datetime.strptime(str(d).replace('-', ''), "%Y%m%d")


In [3]:
d

datetime.datetime(2019, 2, 21, 0, 0)

In [4]:
select_confused_csv = '../confusion_processed/'+str(d.year)+str(d.month).zfill(2)+str(d.day).zfill(2)+'_confusion.csv'
avg_1min = bin_confusion(select_confused_csv)

select_transcript_csv = '../transcripts_processed/' + str(d.year) + str(d.month).zfill(2) + str(d.day).zfill(
    2) + '_transcript_all.csv'

transcript_with_confused_df = confusion2transcript(select_transcript_csv, avg_1min)
confused_transcript, all_transcripts = split_confused_transcripts_by_line(transcript_with_confused_df)
response = send2chatgpt(all_transcripts, confused_transcript)

/Users/lucia/Documents/GitHub/ICSE_2025/notebooks/../app/utils/locate_timings.py:48: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.state[(df.timestamp < f"00:{min}:00")
/Users/lucia/Documents/GitHub/ICSE_2025/notebooks/../app/utils/locate

In [ ]:
transcript_with_confused_df['state']

In [5]:
confused_transcript

"but you've considered it.\nOkay, 45 seconds.\nHow many people have done it? Raise your hand if you've done it.\nOkay, 10 newtons.\nOkay, start passing them in.\nHow many people thought this one was easier than the last one?\nRaise your hand.\nThat's not what we're doing.\nWe just went over the material last lecture.\nIt's easier.\nOkay, got them all, yes?\nThe file is closing in five seconds.\nYes, is there one more?\nOkay, last call, yes?\nOkay, very quickly we get the answer.\nSo, one of the two HCI monikers,\nthe first one being know thy user,\nthe second HCI moniker is user is not like me.\nRight, recognizing that people are different.\nHow many people put user not like me?\nOkay, with surveys.\nIn an online survey, like SurveyMonkey,\nyou're just calling up phone numbers.\nYou generally learn.\nYou know you can call thousands,\nbecause you can blast a meme of a thousand people,\nso hopefully people know that it's a thousands of people,\nbut it would be weird to send a SurveyMonke

In [6]:
f = open('../figure_panels/confused_transcript_0221.txt', 'w')
f.write(confused_transcript)

26833

In [7]:
confused_transcript

"but you've considered it. Okay, 45 seconds. How many people have done it? Raise your hand if you've done it. Okay, 10 newtons. Okay, start passing them in. How many people thought this one was easier than the last one? Raise your hand. That's not what we're doing. We just went over the material last lecture. It's easier. Okay, got them all, yes? The file is closing in five seconds. Yes, is there one more? Okay, last call, yes? Okay, very quickly we get the answer. So, one of the two HCI monikers, the first one being know thy user, the second HCI moniker is user is not like me. Right, recognizing that people are different. How many people put user not like me? Okay, with surveys. In an online survey, like SurveyMonkey, you're just calling up phone numbers. You generally learn. You know you can call thousands, because you can blast a meme of a thousand people, so hopefully people know that it's a thousands of people, but it would be weird to send a SurveyMonkey to just two people you ca